# PostgreSQL connection

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

In [3]:
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')
database = os.getenv('DB_NAME')
table_name = 'raw_data'

In [6]:
filepath = "../data/browser_data.csv"

In [10]:
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

data = pd.read_csv(filepath, dtype='str', sep='^', encoding='utf-8')

data.to_sql(table_name, engine, if_exists='replace', index=False)

print("Success")

Success


# Minio conection

In [10]:
from minio import Minio
from minio.error import S3Error
from dotenv import load_dotenv
import os

load_dotenv()

True

In [11]:
minio_url = os.getenv("MINIO_URL")
access_key = os.getenv("MINIO_USER")
secret_key = os.getenv("MINIO_PASSWORD")

In [13]:
minio_client = Minio(minio_url, access_key=access_key, secret_key=secret_key, secure=False)

In [35]:
def create_bucket(bucket_name: str):
    try:
        if not minio_client.bucket_exists(bucket_name):
            minio_client.make_bucket(bucket_name)
            print("Bucket", bucket_name, "created successfully")
        else:
            print("Bucket", bucket_name, "already exists")
    except S3Error as e:
        print(f"An error occurred: {e}")

def upload_file_to_minio(bucket: str, file_name: str, file_path: str):
    try:
        if not minio_client.bucket_exists(bucket):
            create_bucket(bucket)
        minio_client.fput_object(bucket, file_name, file_path)
        print("File", file_name, "uploaded to bucket", bucket, "successfully")
    except S3Error as e:
        print(f"An error occurred: {e}")

def upload_df_to_minio(bucket: str, file_name: str, df: pd.DataFrame):
    try:
        if not minio_client.bucket_exists(bucket):
            create_bucket(bucket)

        csv_buffer = BytesIO()
        df.to_csv(csv_buffer, index=False, sep='^', encoding='utf-8')
        csv_buffer.seek(0)
        minio_client.put_object(bucket_name=bucket, object_name=file_name, data=csv_buffer, length=len(csv_buffer.getvalue()), content_type="text/csv")
        print("File", file_name,"created from datafra uploaded to bucket", bucket, "successfully")
    except S3Error as e:
        print(f"An error occurred: {e}")

def download_csv_file(bucket: str, file_name: str):
    try:
        response = minio_client.get_object(bucket_name=bucket, object_name=file_name)
        csv_data = BytesIO(response.read())
        
        return csv_data
    except S3Error as e:
        print(f"An error occurred: {e}")
    

In [16]:
create_bucket('raw-data')

Bucket raw-data created successfully


In [20]:
upload_file_to_minio("raw-data", 'raw_data.csv', "../data/browser_data.csv")

File raw_data.csv uploaded to bucket raw-data successfully


In [21]:
import pandas as pd

data = pd.read_csv('../data/html_raw_data.csv', dtype='str', sep='^', encoding='utf-8')

In [32]:
from io import BytesIO

csv_buffer = StringIO()
data.to_csv(csv_buffer, index=False, sep='^', encoding='utf-8')
csv_buffer.seek(0)

0

In [34]:
upload_df_to_minio('test-bucket', 'test_raw_data.csv', data)

File test_raw_data.csv created from datafra uploaded to bucket test-bucket successfully


In [36]:
new_data = download_csv_file('test-bucket', 'test_raw_data.csv')
new_df = pd.read_csv(new_data, dtype='str', sep='^', encoding='utf-8')
new_df.head()

,category,title,price,rating,description,availability
0,Poetry,A Light in the Attic,£51.77,Three,It's hard to imagine a world without A Light i...,In stock (22 available)
1,Historical Fiction,Tipping the Velvet,£53.74,One,"""Erotic and absorbing...Written with starling ...",In stock (20 available)
2,Fiction,Soumission,£50.10,One,"Dans une France assez proche de la nôtre, un h...",In stock (20 available)
3,Mystery,Sharp Objects,£47.82,Four,"WICKED above her hipbone, GIRL across her hear...",In stock (20 available)
4,History,Sapiens: A Brief History of Humankind,£54.23,Five,From a renowned historian comes a groundbreaki...,In stock (20 available)
